When you’re implementing Python programs that handle a non-trivial amount of data,
you’ll eventually see slowdowns caused by the algorithmic complexity of your code. This
usually isn’t the result of Python’s speed as a language (see Item 41: “Consider
concurrent.futures for True Parallelism” if it is). The issue, more likely, is that
you aren’t using the best algorithms and data structures for your problem.

Luckily, the Python standard library has many of the algorithms and data structures you’ll
need to use built in. Besides speed, using these common algorithms and data structures
can make your life easier. Some of the most valuable tools you may want to use are tricky
to implement correctly. Avoiding reimplementation of common functionality will save you
time and headaches.

## Double-ended Queue

The deque class from the collections module is a double-ended queue. It provides
constant time operations for inserting or removing items from its beginning or end. This
makes it ideal for first-in-first-out (FIFO) queues.


In [ ]:
# Preamble to mimick book environment
import logging
from pprint import pprint
from sys import stdout as STDOUT

# Example 1
from collections import deque
fifo = deque()
fifo.append(1)      # Producer
fifo.append(2)
fifo.append(3)
x = fifo.popleft()  # Consumer
print(x)


## Ordered Dictionary

### Ex 2
Standard dictionaries are unordered. That means a dict with the same keys and values
can result in different orders of iteration. This behavior is a surprising byproduct of the
way the dictionary’s fast hash table is implemented.

### Ex 3
The OrderedDict class from the collections module is a special type of
dictionary that keeps track of the order in which its keys were inserted. Iterating the keys
of an OrderedDict has predictable behavior. This can vastly simplify testing and
debugging by making all code deterministic.


In [ ]:
# Example 2
a = {}
a['foo'] = 1
a['bar'] = 2
from random import randint

# Randomly populate 'b' to cause hash conflicts
while True:
    z = randint(99, 1013)
    b = {}
    for i in range(z):
        b[i] = i
    b['foo'] = 1
    b['bar'] = 2
    for i in range(z):
        del b[i]
    if str(b) != str(a):
        break

print(a)
print(b)
print('Equal?', a == b)


# Example 3
from collections import OrderedDict
a = OrderedDict()
a['foo'] = 1
a['bar'] = 2

b = OrderedDict()
b['foo'] = 'red'
b['bar'] = 'blue'

for value1, value2 in zip(a.values(), b.values()):
    print(value1, value2)



## Default Dictionary

### Ex 4
Dictionaries are useful for bookkeeping and tracking statistics. One problem with
dictionaries is that you can’t assume any keys are already present. That makes it clumsy to
do simple things like increment a counter stored in a dictionary.

### Ex 5
The defaultdict class from the collections module simplifies this by
automatically storing a default value when a key doesn’t exist. All you have to do is
provide a function that will return the default value each time a key is missing. In this
example, the int built-in function returns 0 (see Item 23: “Accept Functions for Simple
Interfaces Instead of Classes” for another example). Now, incrementing a counter is
simple

In [ ]:
# Example 4
stats = {}
key = 'my_counter'
if key not in stats:
    stats[key] = 0
stats[key] += 1
print(stats)

# Example 5
from collections import defaultdict
stats = defaultdict(int)
stats['my_counter'] += 1
print(dict(stats))

## Heap Queue

### Ex 7
Heaps are useful data structures for maintaining a priority queue. The heapq module
provides functions for creating heaps in standard list types with functions like
heappush, heappop, and nsmallest.
Items of any priority can be inserted into the heap in any order.

### Ex 8
The resulting list is easy to use outside of heapq. Accessing the 0 index of the heap
will always return the smallest item.

### Ex 9
Calling the sort method on the list maintains the heap invariant.

Each of these heapq operations takes logarithmic time in proportion to the length of the
list. Doing the same work with a standard Python list would scale linearly.

In [ ]:
# Example 6
from heapq import *
a = []
heappush(a, 5)
heappush(a, 3)
heappush(a, 7)
heappush(a, 4)


# Example 7
print(heappop(a), heappop(a), heappop(a), heappop(a))


# Example 8
a = []
heappush(a, 5)
heappush(a, 3)
heappush(a, 7)
heappush(a, 4)
assert a[0] == nsmallest(1, a)[0] == 3


# Example 9
print('Before:', a)
a.sort()
print('After: ', a)



## Bisection

Searching for an item in a list takes linear time proportional to its length when you call
the index method.

The bisect module’s functions, such as bisect_left, provide an efficient binary
search through a sequence of sorted items. The index it returns is the insertion point of the
value into the sequence.

The complexity of a binary search is logarithmic. That means using bisect to search a
list of 1 million items takes roughly the same amount of time as using index to linearly
search a list of 14 items. It’s way faster!


In [ ]:
# Example 10
x = list(range(10**6))
i = x.index(991234)
print(i)

# Example 11
from bisect import bisect_left
i = bisect_left(x, 991234)
print(i)



## Iterator Tools

The itertools built-in module contains a large number of functions that are useful for
organizing and interacting with iterators (see Item 16: “Consider Generators Instead of
Returning Lists” and Item 17: “Be Defensive When Iterating Over Arguments” for
background). Not all of these are available in Python 2, but they can easily be built using
simple recipes documented in the module. See help(itertools) in an interactive
Python session for more details.

The itertools functions fall into three main categories:
[Linking iterators together]
• chain: Combines multiple iterators into a single sequential iterator.
• cycle: Repeats an iterator’s items forever.
• tee: Splits a single iterator into multiple parallel iterators.
• zip_longest: A variant of the zip built-in function that works well with
iterators of different lengths.

[Filtering items from an iterator]
• islice: Slices an iterator by numerical indexes without copying.
• takewhile: Returns items from an iterator while a predicate function returns
True.
• dropwhile: Returns items from an iterator once the predicate function returns
False for the first time.
• filterfalse: Returns all items from an iterator where a predicate function
returns False. The opposite of the filter built-in function.

[Combinations of items from iterators]
• product: Returns the Cartesian product of items from an iterator, which is a
nice alternative to deeply nested list comprehensions.
• permutations: Returns ordered permutations of length N with items from an
iterator.
• combination: Returns the unordered combinations of length N with
unrepeated items from an iterator.

There are even more functions and recipes available in the itertools module that I
don’t mention here. Whenever you find yourself dealing with some tricky iteration code,
it’s worth looking at the itertools documentation again to see whether there’s
anything there for you to use.

In [ ]:
# Example 12
from timeit import timeit
print(timeit(
    'a.index(len(a)-1)',
    'a = list(range(100))',
    number=1000))
print(timeit(
    'bisect_left(a, len(a)-1)',
    'from bisect import bisect_left;'
    'a = list(range(10**6))',
    number=1000))

* Use Python’s built-in modules for algorithms and data structures.
* Don’t reimplement this functionality yourself. It’s hard to get right.